# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**27/05/2020 - C10 S6**

# Computación de alto Rendimiento con Python

Python es utilizado transversalmente, ya sea en la industria o en la academia. Dentro de sus cualidades se encuentra la portabilidad de código, sintaxis intuitiva, disponibilidad de herramientas y documentación. Sin embargo, al ser un lenguaje interpretado se pierden ciertas características intrínsicas de los lenguajes de bajo nivel como C, C++ y Fortran.

Se estudian distintas herramientas para mejorar el rendimiento del interprete CPython, estas se basan en el uso eficiente de objetos base, aplicación de técnicas de paralelismo y compilación utilizando tanto librerías nativas, como desarrolladas por terceros. 

## Perfilamiento y Referenciación

El flujo de trabajo en ciencia de datos consta de numerosas rutinas de carga, procesamiento, visualización y optimización. Estas rutinas son abordadas por medio de técnicas de programación y diseño de código. En este apartado, se debe tener en cuenta la importancia de generar rutinas eficientes, pues significan reducciones en los tiempos de respuesta y uso de recursos. Por lo anterior, es natural que desde fases tempranas del desarrollo de proyectos, se busque optimizar el código. Como directriz general, se recomienda llevar el proceso de desarrollo en dos etapas. La primera consiste en generar rutinas **correctas**, **comprensibles** y **mantenibles**, evitando la sobre-optimización prematura de código. Como segunda etapa, se recomienda comenzar con los procesos de optimización de rutinas. Esto pues, las herramientas que permiten mejorar los aspectos computacionales, interfieren en la sencillez del código, entorpeciendo los procesos de depuración y mantención. 

Una vez que las rutinas están implementadas de manera correcta, la mejor manera de enfocar los esfuerzos, pasa por **perfilar** (*profiling*) las rutinas codificadas. Esto consiste en encontrar las zonas de código criticas en cuanto a carga computacional. La manera más directa de encontrar estas zonas, es por medio del uso de contadores de tiempo o *timers*.

**Ejemplo**

Se utiliza la librería `time` para medir el tiempo de ejecución de una zona de código.

In [ ]:
from math import sin, cos
import time

Se define la función a analizar

In [ ]:
def func_1(a):
    
    result = 0
    for val in a:
        result += sin(val) + cos(val)**2
    return result

Se define un rango de datos a operar y se estudia el tiempo de ejecución por medio de la función `process_time`.

In [ ]:
x = [0.1 * i for i in range(1000)]

t0 = time.process_time()
for r in range(1000):
    func_1(x)
t1 = time.process_time()


print("Tiempo transcurrido", t1 - t0)

**Ejercicio**

1. Defina una clase *context manager* llamada `Timer`. Esta debe abstraer la dinámica de medición temporal anterior. *Hint*: Deberá definir los métodos `__enter__` y `__exit__`, en este último se produce el `print` de tiempo transcurrido.

En algunas ocasiones se desea medir el tiempo de ejecución para tareas sencillas, la librería estándar de Python provee el módulo `timeit`, este puede ser utilizado directamente en la consola interactiva IPython o en notebooks de Jupyter por medio del comando mágico `%timeit`. 

**Ejemplo**

Se mide el tiempo de ejecución promedio para la función coseno de NumPy.

In [ ]:
import numpy as np

In [ ]:
%timeit np.cos(0.5)

Se compara con la función del módulo `Math`

In [ ]:
%timeit cos(0.5)

Se puede ver una gran diferencia en los tiempos de ejecución promedio. 

**Ejercicio**

1. Diferencie los comandos `%time`, `%timeit`, `%%timeit`.
2. En Pyhton existen 2 maneras para declarar listas, tuplas y diccionarios, una de ellas es la *forma funcional* y viene dada por la función de conversión de tipo de datos, por ejemplo para diccionarios es `dict()`. La otra forma es utilizar *expresiones literales* esto quiere decir declarar listas usando `[]`, tuplas usando `()` y diccionarios / conjuntos usando `{}`. Una de estas dos maneras es más rápida que la otra, utilice `%timeit` para deducir cual. 
3. ¿Qué operación es más eficiente ?

    1. `list(range())`
    2. `[*range()]`

Un **perfilador**  (*profiler*) es un programa que ejecuta una rutina y monitorea las funciones ahí especificadas, obteniendo métricas de rendimiento como el consumo de tiempo y memoria. Por otra parte, la referenciación (*benchmarking*) consiste en extraer zonas de código de interés para probar su rendimiento antes y después de aplicar técnicas de optimización. IPython provee de un perfilador de código dado por la orden `%prun`.

**Ejemplo**

1. Se perfila una función utilizando `%prun`. En primera instancia se define tal función

In [ ]:
def benchmark_sum(n):
    '''Funcion de referencia que suma n elementos transformados
    '''
    ac = 0
    
    for i in range(n):
        to_sum = [(i // 2)**n + (i-n)**(n // 3) for i in range(n)]
        ac = sum(to_sum)
    
    return ac

Se perfila la función con `%prun`

In [ ]:
%prun benchmark_sum(500)

EL resultado corresponde las mediciones temporales de cada función involucrada en la ejecución de `benchmark_sum(500)`. En este caso, la mayoria del tiempo se utiliza en la ejecución de la compresión de listas `<listcomp>`. Esto indica que la mejor manera de optimizar el código de `benchmark_sum`pasa por optimizar tal sección del código.

Con `%lprun` es posible perfilar por linea de código, para ello es necesario instalar el módulo `line_profiler`. 

**Ejemplo**

Se carga la extensión `%lprun` y se prueba con `benchmark_sum`.

In [ ]:
%load_ext line_profiler

El comando `%lprun` toma como parámetro una orden de Python y su principal argumento. Las funciones que se desean perfilar deben ser especificadas de manera explicita con la orden `-f`. En el caso de `benchmark_sum` esto se haría según el siguiente código

In [ ]:
%lprun -f benchmark_sum benchmark_sum(500)

El resultado es una tabla con el tiempo utilizado en cada linea de las funciones perfiladas, mostrando porcentajes del tiempo consumido en cada paso.

**Ejercicio**

1. Los perfiladores `%prun` y `%lprun` tienen en común los argumentos -D, -T y -r utilice el parámetro `?` para investigar estas opciones.

Es posible perfilar uso de memoria, para ello existen los comandos mágicos `%memit` y `%mprun`. Para utilizarlos es necesario instalar el módulo `memory_profiler` y cargarlo mediante

```python
%load_ext memory_profiler
```

**Ejemplo**

Se carga la extensión y se perfila el uso de memoria para `benchmark_sum`

In [ ]:
%load_ext memory_profiler

En primera instancia perfilamos utilizando la linea mágica `%memit`, la cual es equivalente a `timeit` pero ofrece medidas sobre el uso de memoria. 

In [ ]:
%memit benchmark_sum(500)

Se puede observar un uso de memoria en torno a 90 MB.

De manera análoga a `%prun` la librería `memory_profiler` permite utilizar `%mprun`, con la cual se pueden obtener descripciones linea a linea del uso de memoria. El uso de este comando es un poco más restrictivo, pues solo permite medir funciones definidas en módulos (no dentro de un notebook). Para ello, se crea el módulo `memory_demo`. La manera sencilla de hacer esto, es mediante el comando mágico `%%file` este permite crear archivos en el directorio de trabajo actual, utilizando el código dentro de una celda de jupyter. 

Se procede a generar el módulo que contiene el código de `benchmark_sum` utilizando el comando `%%file`.




In [ ]:
%%file bench_module.py

def benchmark_sum(n):
    '''Funcion de referencia que suma n elementos transformados
    '''
    ac = 0
    
    for i in range(n):
        to_sum = [(i // 2)**n + (i-n)**(n // 3) for i in range(n)]
        ac = sum(to_sum)
    
    return ac

a continuación, se importa el módulo creado y se perfuila su memoria mediante:

In [ ]:
from bench_module import benchmark_sum
%mprun -f benchmark_sum benchmark_sum(500)

El resultado se muestra en pantalla, obteniendo detalles linea a linea.

**Ejercicio**

1. Agregue la linea `del to_sum` luego de ejecutar `ac = sum(to_sum)` y antes de terminar el ciclo `for` principal. Estudie el efecto en el consumo de memoria en la función.

## Optimización con Código Nativo

Una de las manera más eficientes de mejorar el rendimiento de aplicaciones es por medio del uso de algoritmos más eficientes en conjunción de estructuras de datos mejor diseñadas. A continuación, se estudian los algoritmos y estructuras de datos presentes de manera nativa en Python que permiten acelerar ciertas rutinas. 

En términos generales, lso algoritmos pueden ser clasificados según su *complejidad computacional*, esta clasificación se expresa según la notación de O-grande, que corresponde a una cota superior de las operaciones requeridas para ejecutar una tarea.

Si la tarea no depende del tamaño del input (acceder a cierta llave de un diccionario por ejemplo) se dice que el algoritmo asociado se efectúa en tiempo constante, denotado por $O(1)$. Esto quiere decir, que sin importar la cantidad de datos disponibles, el tiempo de ejecución de la tarea será siempre el mismo. 

**Ejemplo**

Se genera un lista, a cada uno de sus elementos se le realiza una operación básica.

In [ ]:
lista = list(range(10))

for i in range(len(lista)):
    lista[i]+=100

En este algoritmo, la operación `lista[i]+=100` es repetida tantas veces como elementos hay en `lista`, que corresponde al tamaño de los datos de entrada. Al observar la que las operaciones realizadas por este algoritmo son proporcionales a la cantidad de elementos de `lista`, se puede decir que su tiempo de ejecución es $O(N)$ donde `N = len(lista)`. 

### Optimización de Operaciones con Listas

Las listas de Python son colecciones ordenadas de elementos, estás se encuentran clasificadas como *arreglos*, que a la vez corresponden a una estructura de datos caracterizada por contener elementos contiguos en bloques de memoria, cada uno de los cuales contienen una referencia a un objeto de Python. La ventaja de las listas recae en la facilidad que entregan par acceder, modificar y agregar elementos. Dado que acceder y modificar elementos de una lista corresponde a acceder a espacios de memoria que a priori no dependen de la longitud de la lista, se dice que estas operaciones tienen complejidad $O(1)$. Por otra parte, para agregar un elemento a una lista por medio de `.append()`, puede requerirse re-ubicar la memoria del arreglo asociado, operación que toma un tiempo de $O(N)$. Sin embargo, tal operación es muy poco frecuente, pues por lo general se tiene acceso a bloques de memoria contiguos, por tal motivo, se dice que la operación `.append()` tiene un tiempo esperado de ejecución de $O(1)$. 

Para agregar o eliminar datos al inicio de un arreglo, se requiere hacer una traslación (o *shift*) de los demás elementos por lo que tal operación toma un tiempo de $O(N)$. Para agregar o remover elementos de un arreglo en una posición distinta a la última, se opera de manera análoga. 

**Ejemplo**

Se definen listas para estudiar la complejidad de ciertos métodos empíricamente. Se definen los parámetros

In [ ]:
n_0, n_1, n_2 = (int(10e5), int(5*10e5), int(10e6))

Se generan una funciones de referencia

In [ ]:
def copy_objs(obj_0,obj_1,obj_2):
    '''Abstraccion auxiliar para copiar elementos.'''
    return (obj_0.copy(),obj_1.copy(),obj_2.copy())

def bench_pop(l_0,l_1,l_2, index = -1):
    '''Funcion de referncia para eliminacion de elementos.'''
    
    l_0.pop(index)
    l_1.pop(index)
    l_2.pop(index)
    

def bench_append(l_0,l_1,l_2, index = 1):
    '''Funcion de referencia para insertar 1 con append.'''
    
    l_0.append(index)
    l_1.append(index)
    l_2.append(index)
    

def bench_insert(l_0,l_1,l_2, index = (0,1)):
    '''Funcion de referncia para insertar 1 con insert.'''
    
    l_0.insert(*index)
    l_1.insert(*index)
    l_2.insert(*index)

Se construye el test 

In [ ]:
lista_0, lista_1, lista_2 = (list(range(n_0)), list(range(n_1)),
                             list(range(n_2)))

Se elimina el ultimo elemento

In [ ]:
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo constante
%lprun -f bench_pop bench_pop(l_0,l_1,l_2)

Se elimina el primer elemento

Se elimina el primer elemento

In [ ]:
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo lineal
%lprun -f bench_pop bench_pop(l_0,l_1,l_2,0)

Se inserta 1 en la ultima posicion

In [ ]:
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo constante (casi seguramente)
%lprun -f  bench_append bench_append(l_0,l_1,l_2)

Se inserta 1 en la primera posicion

In [ ]:
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo lineal
%lprun -f bench_insert bench_insert(l_0,l_1,l_2)

Para efectuar inserciones de manera eficiente (siempre en tiempo constante) Se puede utilizar la estructura de datos `deque` del módulo `collections`. Estas estructuras se comportan como listas, están diseñadas para acelerar la inserción de objetos y añaden los métodos `.popleft` y `.appendleft`. 

**Ejemplo**

Se compara `.popleft` en *deques* con `.pop(0)` en listas.

In [ ]:
from collections import deque

def bench_pop_left(d_0,d_1,d_2):
    '''Funcion de referncia para eliminacion de elementos.'''
    
    d_0.popleft()
    d_1.popleft()
    d_2.popleft()
    
def bench_append_left(d_0,d_1,d_2, val = 1):
    '''Funcion de referncia para insertar 1 con insert.'''

    d_0.appendleft(val)
    d_1.appendleft(val)
    d_2.appendleft(val)

Se definen los objetos sobre los que se trabajará

In [ ]:
deque_0, deque_1, deque_2 = tuple(map(deque, [lista_1, lista_1, lista_2]))

In [ ]:
d_0, d_1, d_2 = copy_objs(deque_0,deque_1,deque_2)

# Se observa un tiempo constante
%lprun -f bench_pop_left bench_pop_left(d_0, d_1, d_2)

In [ ]:
%%timeit 
d_0, d_1, d_2 = copy_objs(deque_0,deque_1,deque_2)

Copiar objetos de tipo `deque` tiene una carga de aproximadamente 190 ms

In [ ]:
%%timeit 
d_0, d_1, d_2 = copy_objs(deque_0,deque_1,deque_2)
bench_pop_left(d_0, d_1, d_2)

Por su parte, aplicar el benchmark `bench_pop_left` tarda en promedio 191 - 190 ms = 1 ms.

En cuanto a las listas, la operación de copiar lleva unos 112 ms en promedio

In [ ]:
%%timeit 
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

Aplicar el benchmark `bench_pop` en listas lleva un tiempo promedio de 120 ms - 112 ms = 8ms. Por lo que se aprecia un aumento en el rendimiento. Cabe señalar que tal aumento se ve sujeto a una carga mayor en el proceso de copia de objetos, por tal motivo, vale la pena evitar la copia de objetos tipo `deque` y utilizarlos para acceder a lista con una gran cantidad de elementos.

In [ ]:
%%timeit 
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)
bench_pop(lista_0,lista_1,lista_2,0)

**Ejercicios**

1. Repita el proceso de comparación para `.appendleft` en deques  e `.insert()` en listas. 

2. Cual es la complejidad computacional de acceder a:
    1. Primer elemento de un deque / lista
    2. Ultimo elemento de un deque / lista
    3. Un elemento distinto del último o el primero (ej: el elemento de la mitad de un arreglo).

3. El módulo `bisect` permite hacer búsquedas rápidas en arreglos ordenados. la función `bisect.bisect` permite encontrar el índice en cual insertar un elemento, manteniendo el orden del arreglo operado. 
    1. Genere la lista ordenada 'ordered_list'  de números entre 0 y 10.
    2. Elimine el cuarto elemento de la lista, guarde su valor en la variable `dropped`. Se debe hacer en una linea de código. 
    3. Importe el módulo `bisect` y utilice el comando `bisect.bisect(ordered_list,dropped)`. ¿Qué significa el valor retornado por la función?¿qué operación se efectúa por medio del comando recién aplicado?
    4. Haga un código de referencia para comparar las funciones `list.index()` y `bisect.bisect()` por medio de perfilamiento temporal. Para comprobar sus resultados utilice el hecho de que  el tiempo de ejecución para `bisect.bisect` es de $O(\log(N)$, mientras que el de `list.index()` es de $O(N)$. 
    5. Estudie la función `bisect.bisect_left`. 
    

### Optimización de Operaciones con Diccionarios

La gran flexibilidad de los diccionarios los hacen un objeto central en el uso de Python. Estos son implementaciones de *hash maps*, es decir, son estructuras de datos construidas por medio de asociaciones *llave - valor*, donde a cada llave, se asigna un índice especifico, de tal manera que el valor de tal índice puede ser ordenado en un arreglo. Por tal motivo, los diccionarios son altamente eficientes en procesos de eliminación, acceso e inserción teniendo un tiempo promedio de ejecución de $O(1)$. 

Para acceder a los índices dados por el *hash map* se puede utilizar la función `hash` de Python, esta opera sobre distintos tipos de datos.

**Ejemplo**

Se aplica `hash` a diferentes objetos

In [ ]:
print('hash string: ',hash('MA6202'))
print('hash int:' , hash(1234))
print('hash tuple', hash(('a','b','c')))

Un objeto puede ser operado por `hash` (*hashable object*) si tiene un método `__hash__` y puede ser comparado por medio de `__eq__` por ejemplo. Si un objeto es *hashable* significa que puede ser utilizado como llave de un diccionario, en general, todos los objetos inmutables de Python son *hashables* mientras que las listas y diccionarios, por ser inmutables, no lo son.  

**Ejemplo**

Es posible usar las ventajas de accesibilidad de diccionarios agrupar listas de manera eficiente. Para esto se utiliza el objeto `defaultdict` de la librería `collections`

In [ ]:
from collections import defaultdict

Se construye la lista a agrupar

In [ ]:
to_group = [('a', 1), ('b', 2), ('c', 3), ('b', 4), ('d', 1)]

Los objetos `defaultdict` son subclases de `dict`. Reciben como argumentos un valor inicial para su el atributo `.default_factory`, cuyo valor por defecto es `None`. 

Los objetos `defaultdict` poseen todas las funcionalidades de un diccionario pero añaden el método `.__missing__()` con el cual se proveen valores por defecto, los cuales se asignan a una nueva llave, es decir, permiten inicializar diccionarios entregando solo el valor de la llave (y no su valor asociado), pues a cada llave nueva, se asigna un valor por defecto de manera automática. 

Según lo enterior, inicializar un objeto `defaultdict` por medio de `defaultdict(list)` genera un diccionario, en el cual, cada llave nueva tendrá asociada una lista vacía (valor por defecto del atributo `.default_factory` para este tipo de dato).

In [ ]:
D = defaultdict(list)

Una vez teniendo el diccionario definido, es posible agrupar los elementos de la lista `to_group` y se perfila por medio de:

In [ ]:
%%timeit
D = defaultdict(list)

for k, v in to_group:
    D[k].append(v)

sorted(D.items())

El código anterior genera las llaves `k`, que por defecto poseen una lista vacía asociada, a cada lista vacía agregan por medio de `append` (tiempo de ejecución constante) el elemento inspeccionado `v`. 

Se implementa la misma funcionalidad usando ciclos `for` y `append` de listas, se perfila utilizando `%%timeit`

In [ ]:
%%timeit
L = []
for elem in to_group:    
    if len(L) == 0:
        L.append((elem[0],[elem[1]]))
    else:
        c = 0
        for l in L:
            if l[0] == elem[0]:
                l[1].append(elem[1])
                c = 1
        if c == 0:
             L.append((elem[0],[elem[1]]))

L

Con lo anterior apreciamos una ganancia en eficiencia, que tambien se traduce en simpleza.

**Ejercicios**

Los objetos `defaultdict` permiten además aumentar la eficiencia al momento de contar elementos de un arreglo. Para ver esto, implemente una función que:

1. Reciba una *iterable* como argumento.
2. Inicalice un objeto `defaultdict` con tipo de dato `int`. ¿Qué valor se asocia por defecto?.
3. Recorra cada elemento del iterable, registrando su número de ocurrencias en una llave del objeto `defaultdict` antes inicializado.
    

El módulo `collections` permite implementar el procedimiento del ejercicio 2 anterior por medio de la clase `Counter`

**Ejemplo**

Se cuentan los elementos de una lista por medio de la clase `Counter`

In [ ]:
from collections import Counter
import numpy as np

lista = np.random.randint(0, 10, size=100)
counts = Counter(lista)

sorted(counts.items())

**Obs**: El método de counteo por medio de la clase `Counter` tiene un tiempo de ejecución de $O(N)$.

Otra ventaja de los diccionarios es que permiten buscar palabras de manera rápida en una lista de documentos, en este caso, tal lista de documentos viene representada dada por:

In [ ]:
with open('text.txt','r') as file:
    lines = file.readlines()
    lines = [l.rstrip(' \n') for l in lines]
        
lines

supongamos que se busca la palabra 'imaginario' en cada documento, es posible generar una lista de documentos con tal palabra por medio de:

In [ ]:
to_search = 'imaginario'
%timeit found = [line for line in lines if to_search in line]

Se debe considerar que el tiempo de ejecución asociado a consultar por una palabra es $O(N)$. Para mejorar esto, se puede construir un diccionario, donde a cada palabra se asocie un índice, donde este último corresponde al la linea (o documento si se prefiere) al que pertenece. Esto se puede hacer mediante el siguiente código

In [ ]:
index = defaultdict(list)

for i, line in enumerate(lines):
    
    for word in line.split():
        index[word].append(i)

En el diccionario generado, hacer búsquedas es de orden $O(1)$, luego para la misma consulta antes hecha se tiene

In [ ]:
%%timeit
res = index[to_search]
[lines[i] for i in res]

Es decir, un aumento substancial de rendimiento. Cabe mencionar, que este procedimiento solo tiene sentido si se busca hacer una cantidad alta de consultas sobre un arreglo de lineas/documentos, esto pues, el tiempo de preprocesamiento para generar el indexado por diccionario puede ser muy alto.

### Optimización de Operaciones con Conjuntos

A diferencia de las listas, los conjuntos son colecciones no ordenadas, donde cada elemento debe ser único. La implementación de conjuntos en Python sigue la misma lógica de los diccionarios en cuanto ambos utilizan funciones *hash*. Por tal motivo, en conjuntos se tienen operaciones rápidas para añadir, eliminar y acceder a elementos. (Del orden $O(1)$).

**Ejercicio**

Los tiempos de ejecución para los métodos `A.union(B)`, `A.intersection(B)` y `A.difference(B)` son $O(a+b)$, $O(\min(a,b))$ y $O(a)$, donde $a = |A|$ y $b = |B|$. 

1. Construya una función de referencia para cada método y utilice un perfilamiento adecuado para comprobar la afirmación anterior de manera empírica. 

Se puede hacer uso de conjuntos para efectuar consultas rápidas.

**Ejemplo**

Se utiliza el objeto `index` creado para hacer búsquedas sobre texto. Se consulta sobre los documentos donde las palabras 'imaginario' y 'vive' ocurren simultáneamente, se obtiene un estimado del tiempo de ejecución.

In [ ]:
to_search = ['imaginaria', 'vive']

In [ ]:
%%timeit

res_0 = [lines[i] for i in index[to_search[0]]]
res_1 = [lines[i] for i in index[to_search[1]] ]

[r for r in res_1 if r in res_0]

Se puede modificar la función de indexación para que opere sobre conjuntos.

In [ ]:
index_set = defaultdict(set)

for i, line in enumerate(lines):
    
    for word in line.split():
        index_set[word].add(i)

luego se hace la misma búsqueda por medio de

In [ ]:
%%timeit
index_set[to_search[0]].intersection(index_set[to_search[1]])

En este apartado se ve un incremento substancial.

### Optimización con uso de Memoización

También se puede mejorar el rendimiento de aplicaciones por medio de un uso eficiente de la memoria, una de las ideas tras esta premisa es la de guardar los resultados de operaciones intensivas en un espacio de memoria llamado *cache*, este espacio puede estar ubicado en memoria (RAM), disco o almacenada de manera remota. El acto de guardar resultados en memoria para luego utilizarlos de manera directa se denomina **memoización**. y es una forma de *chaching* o uso de memorias *cache*.

Python ofrece el decorador `@lru_cache` accesible desde la librería base `functools`. Este decorador puede ser utilizado de manera sencilla para guardar resultados en memoria y luego accederlos. 

**Ejemplo**

Se utiliza el decorador `@lru_cache` sobre una función sencilla. En primera instancia se importa el módulo necesario.

In [ ]:
from functools import lru_cache

Se define la función a memoizar y se aplica el decorador

In [ ]:
@lru_cache
def simple_func(x, y):
    '''Funcion de prueba para memoizar.'''

    print('Obteniendo el resultado...')

    return x**y + y**x

Para comprobar el funcionamiento del decorador se llama la función dos veces sobre el mismo argumento.

In [ ]:
args = (2,5)
simple_func(*args)

Se repite el procedimiento 

In [ ]:
simple_func(*args)

En este último caso se observa que el resultado es obtenido directamente desde la memoria. 

**Ejercicios**

`@lru_cache` es un decorador que acepta argumentos de entrada, permitiendo el uso del `max_size`. Con este parámetro se especifica el tamaño máximo de memoria para el cache asociado a la función. 

1. Decore la función anterior indicando como parámetro `max_size = 8`.

2. ¿Qué ocurre cuando se llena el tamaño maximo y se realizan más cálculos? *Hint*: lru significa *least recently used*.

3. Acceda a la información del *cache* por medio del método `.cache_info()` de la función decorada. ¿Qué significa *hit* y *miss* en este contexto?


Un ejemplo más avanzado es el de memoizar funciones recursivas

**Ejemplo**

Se trabaja con la función `factorial` almacenando sus resultados en *cache*.

In [ ]:
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

Se mide el tiempo de ejecución para `n=1000`.

In [ ]:
%%timeit
factorial(1000)

Se memoiza la función y se repite el experimento

In [ ]:
@lru_cache
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

In [ ]:
%%timeit
factorial(1000)

Con lo que se comprueba la eficiencia del método. 

**Ejercicios**

1. Programe secuencia de Fibonacci, perfile su consumo de tiempo y luego compare con una versión memoizada.

2. Instale el módulo `joblib`. Este módulo permite guardar resultados en disco por medio del objeto `Memory`. Utilice el decorador `@memory.cache` para memoizar as funciones anteriores (factorial y fibonacci). Compare los tiempos de ejecución al guardar los resultados en disco vs ram.

**Obs**: La ventaja de utilizar técnicas de *caching* tienen un costo, este radica en aumentar el consumo de memoria, si esta memoria esta localizada en disco, el acceso puede ser muy lento y el rendimiento puede decaer drásticamente. Antes de usar este tipo de estrategias, se recomienda estudiar la factibilidad, teniendo en cuenta las politicas de almacenamiento y acceso de los resultados y su relación con el rendimiento del programa que se desea implementar.

### Optimización con uso de Compresiones y Generadores

Las compresiones de lista están altamente optimizadas en Python y por tanto puede ser utilizadas para reemplazar ciclos `for` en ciertas circunstancias. 

**Ejemplo**

Es posible ganar un mayor rendimiento al utilizar comprensión de listas en vez del ciclo `for` en el siguiente código

In [ ]:
n = int(10e4)

def for_loop(n=n):
    res = []
    for i in range(n):
        res.append(i*(i+1) - i**2)
    
    return sum(res)/n

Se mide su consumo de tiempo por medio de `%%timeit`

In [ ]:
%%timeit
for_loop()

Se implementa la misma función haciendo uso de compresión de listas y de diccionarios.

In [ ]:
def list_comp(n=n):
    return sum([i*(i+1) - i**2 for i in range(n)])/n
    
def dic_comp(n=n):
    return sum({i: i*(i+1) -i**2 for i in range(n)})/n

In [ ]:
%%timeit
list_comp()

In [ ]:
%%timeit
dic_comp()

En el caso de compresión de listas vemos una mejoría, por su parte, en compresión de diccionarios, podemos esperar que operaciones de reducción sean más lentas pues se hace uso de llaves. 

En términos de memoria, cada compresión de listas (o diccionarios) ocupa un nuevo espacio, lo cual aumenta el uso de memoria. Para atacar este problema, se puede hacer uso de **generadores**. 

Un generador es un iterable que guarda que posee memoria solo de su estado actual y una regla de cambio para el estado siguiente. 

**Ejemplo**

La función `map` toma como argumentos una función y un iterator, el resultado de su aplicación es un generador. Para estudiar el comportamiento de este tipo de objetos se construyen dos funciones y se perfilan ...



In [ ]:
# %%file bench_module.py -a

def list_comp_list(n=int(10e6)):
    '''Concatena operaciones sobre comprensiones de lista'''

    l_1 = [i**2 for i in range(n) if i % 2 == 0]
    l_2 = [i * (i - 1) for i in l_1]

    l_3 = [i // 3 for i in l_2]

    return max(l_3) / n**2

se construye la misma función utilizando generadores `map`

In [ ]:
# %%file bench_module.py -a

def list_comp_map(n=int(10e6)):
    '''Concatena operaciones sobre comprensiones de lista'''

    l_1 = map(lambda i: i**2, [i for i in range(n) if i % 2 == 0])
    l_2 = map(lambda i: i * (i - 1), l_1)
    
    l_3 = map(lambda i: i // 3, l_2)

    return max(l_3) / n**2

se perfila la memoria utilizada por estas funciones

In [ ]:
%memit list_comp_list()

In [ ]:
%memit list_comp_map()

**Ejercicios**

Se pueden entender los objetos generadores con las estructuras ya vistas. El objetivo de estos ejercicios es revisar ciertos aspectos de interés al usar generadores.

1. Se puede definir un generador por medio de *compresnsión de generadores* este tipo de comprensión sigue la misma sintaxis que una compresión usua (listas o diccionarios) solo que se utilizan paréntesis normales (como una '*compresion de tuplas*'). Defina un generador por medio de *compresión de generadores*.

2. Se pueden definir generadores a partir de funciones, en este caso, el comando `return` debe ser sustituido por `yield`. Este comando permite que el generador definido sea un iterable, el cual solo guarda su estado actual de ejecución y posee una *regla de transición* dada por el cuerpo de la función. Implemente un generador de secuencias inifinitas. Para ello:
    1. Defina una función `infinite_seq`, que no tiene argumentos de entrada. 
    2. Defina un acumulador de suma por medio de la variable `sum = 0`.
    3. Defina un bloque `while True` dentro de este inserte las ordenes `yield sum` y en la linea siguiente `sum +=1`. 
    4. Itere sobre su generador utilizando el método `next`. Interprete la función de `yield` en el bloque anterior. 

## Uso eficiente de Arreglos con Numpy y Pandas

### Uso eficiente con NumPy

Como ya se ha discutido, NumPy provee una rutinas altamente eficientes para realizar operaciones matemáticas complejas basándose en arreglos de C y FORTRAN. Esto permite tener velocidades optimas aún cuando Python sea interpretado. Otra de las cualidades de NumPy es que almacena resultados intermedios en memoria, es posible mejorar tal aspecto por medio del paquete `numexpr` , el cual permite optimizar y compilar arreglos de manera rápida.

**Ejemplo**

se definen arreglos de NumPy

In [ ]:
import numpy as np

In [ ]:
n = int(10e7)
x, y, z = (np.random.rand(n), np.random.rand(n), np.random.rand(n))

Para obtener las ventajas de `numexpr` es recomendable trabajar con arreglos de gran tamaño. A continiación se utiliza la función `numexpr.evaluate` para procesar una operación entre los arreglos definidos, esta función actúa como `eval` de Python, por lo que recibe un string y lo ejecuta.

In [ ]:
from numexpr import evaluate

In [ ]:
%%timeit
evaluate('x + z*y**2')

In [ ]:
%%timeit 
x + z*y**2

**Ejercicio**

1. Estudie la documentación de `numexpr` y busque las funciones soportadas por `evaluate`.

2. Genere una matriz de distancia entre dos arreglos de NumPy. Esta contiene la distancia euclidiana en cada una de sus componentes. Compare los tiempos de ejecución de obtener tal matriz con solamente con expresiones de NumPy y utilizando `numexpr`. 

3. Estudie la función `numexpr.set_num_threads()`

Como consideraciones generales al trabajar con NumPy se puede tener en cuenta:

1. Las operaciones *inplace* son más rápidas que sus contrpartes.

se define una operacion *inplace* y se compara con una asignación

In [ ]:
n = int(5*10e5)

In [ ]:
%%timeit
a = np.random.rand(n)
a *= 5

In [ ]:
%%timeit
a = np.random.rand(n)
b = a * 5

2. Aplicar *reshape* no implica generar una copia, por su parte, trasponer si. Por lo anterior, trasponer utiliza más memoria que cambiar la forma de un arreglo. Vale la pen

3. `flatten()` y `ravel()` permiten cambiar la forma de un arreglo a dimensión 1, sin embarog, `flatten()` retorna una copia, mientras que `ravel()` solo lo hace si es necesario. Por tal motivo `ravel()` es más rápido con arreglos de gran tamaño.

In [ ]:
n = int(100)

In [ ]:
%%timeit
a = np.random.random(size = [n,n])
b = a.flatten()

In [ ]:
%%timeit
a = np.random.random(size = [n,n])
b = a.ravel()

4. Seimrpe mejor utilizar [reglas de broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) para operar con arreglos de distinto tamaño, es decir, se debe prevenir el uso de `reshape` si es que dos arreglos son compatibles según alguna regla de *broadcasting*. A modo de ejemplo, se estudia el producto externo entre dos arreglos.

In [ ]:
n = int(1000)

In [ ]:
a = np.random.random(size = n)
a_1 = a[:,np.newaxis]
a_2 = a[np.newaxis, :]

'''
Obs: np.tile permite copiar un arreglo segun un patron (rep_f,rep_c)
de esta forma se repite tantas veces por fila como rep_f y tantas
veces por columna como rep_c.
'''

%timeit np.tile(a_1, (1, n)) * np.tile(a_2, (n, 1))

In [ ]:
%timeit a_1 * a_2

5. En arreglos de gran tamaño, donde la precisión no es un problema, se puede reducir el consumo de memoria al disminuir la presición numérica de los elementos del arreglo.

In [ ]:
a.dtype

In [ ]:
b = a.astype('float16')
b.dtype

**Ejercicio**

Perfile el uso de memoria y velocidad de ejecución para arreglos de punto flotante de doble presición  `float64`versus punto flotante de presición singular `float32`.

Otro aspecto a optimizar de Python es la velocidad de los ciclos `for`. Por lo general, estos bloques son inherentemente lentos. 

**Ejemplo**

Se construye una función que calcula ciertas operaciones sobre un arreglo.

In [ ]:
def slow_for(n=1000):
    arr = np.arange(n)
    diff = np.zeros(n-1)
    
    for i in range(1, n):
        diff[i-1] = np.sqrt(arr[i]**2 - arr[i-1]**2)
    
    return sum(diff)/n

se estudia su tiempo de ejecución

In [ ]:
%timeit slow_for()

Para mejorar la velocidad, se puede utilizar la **vectorización** que entrega el broadcasting de arreglos. 

In [ ]:
def for_less(n=1000):
    arr = np.arange(n)
    diff = np.sqrt(arr[1:]**2 - arr[:-1]**2)
    
    return sum(diff)/n

se estudia su tiempo de ejecución

In [ ]:
%timeit for_less()

Por le general se recomienda cambiar todo ciclo `for` por la versión vectorizada de NumPy (que hace uso de broadcasting de arreglos).

### Uso eficiente con Pandas

Las directrices de manejo eficiente de la librería Pandas, pasan por utilizar métodos de NumPy siempre que sea posible, preferir funciones de agregación de datasets nativas como `merge` y `concat` sobre agregaciones por medio de ciclos `for`. 

**Ejemplo**

Se carga un dataset consistente en 24 archivos con información sobre criptomonedas, la primera imeplemtación cosiste en generar un dataset vacío, para completarlo según un clico `for`

In [ ]:
import pandas as pd
import glob

In [ ]:
%%timeit -r 10
df_1 = pd.DataFrame()
for file in glob.glob('data/*.csv'):
    df_1 = df_1.append(pd.read_csv(file))

Para mejorar el rendimiento, se puede aplicar el método `concat` sobre un generador que sintetice el funcionamiento del ciclo `for`.

In [ ]:
%%timeit -r 10
generator = map(pd.read_csv, glob.glob('data/*.csv'))
df = pd.concat(generator)

Donde se aprecia una mejora considerable. 

#### Optimizaciones de indexado

Pandas ofrece operaciones optimizadas de indexado, estas permiten fusiones de datasets y búsquedas eficientes. En este caso, se debe considerar que fusionar datasets por medio de `merge` es más eficiente cuando se hace utilizando indices.

**Ejemplo**

El dataset `df_comp` contiene información complementarias a `df` donde según el símbolo asociado a cada criptomoneda, se agrega información sobre la variación porcential en las últimas 1, 2 horas y ultimos 7 dias.

In [ ]:
generator = map(lambda x: pd.read_csv(x, index_col=0), glob.glob('data/*.csv'))
df = pd.concat(generator)
df.head()

In [ ]:
df_comp = pd.read_csv('data/comp_data/comp_data.csv', index_col= 0)
df_comp.head()

Se procede a fusionar los datasets según la columan `Simbol`

In [ ]:
to_merge = df.copy()

In [ ]:
%%timeit -r 10
df_m = to_merge.merge(df_comp, on ='Simbol')

Se procede a indexar por `Simbol` 

In [ ]:
to_add = df_comp.copy()

to_merge.set_index('Simbol')
to_add.set_index('Simbol')

In [ ]:
%%timeit -r 10
df_m1 = to_merge.merge(to_add, left_index=True, right_index=True)

con lo que se observa cierta mejora en el rendimiento. En general, al trabajar con datasets de mayor tamaño, esta diferencia en redimiento comienza a ser más notoria. 


En cuanto al acceso a elementos de un dataframe siempre es más rápido utilizar `iloc` para acceder a arreglos de indices:

In [ ]:
# se usa este tipo de merge para conservar los indices iniciales
df_m = to_merge.merge(df_comp, on ='Simbol')

In [ ]:
%%timeit -r 10
# Version sin loc
df_m['Volum 24 hores'][[500,400,300,100,50,1]]

In [ ]:
%%timeit -r 10
# Version loc
df_m.loc[[500,400,300,100,50,1],'Volum 24 hores']

In [ ]:
%%timeit -r 10
#Version .iloc
df_m.iloc[[500,400,300,100,50,1],5]

**Ejercicio**

1. Para acceder a registros de un dataframe de manera individual (no consultando por arreglos de registros) existen los métodos `at` e `iat`, análogos a `.loc` e `.iloc` respectivamente. Compare sus rendimientos. Observe que tanto `at` como `iat` realizan inferencia de tipos de datos, pruebe acceder a un valor de un dataframe por medio de `at` y luego asigne un valor con un tipo de dato distinto, ¿qué ocurre?

Por lo general, se recomienda evitar utilizar ciclos `for` para iterar sobre objetos de Pandas, pues acceder a valores dentro de este tipo de estructuras tiene un alto costo computacional asociado. En este contexto, se busca vectorizar las operaciones. Cuando no es posible vectorizar, existen ciertos métodos alternativos con los cuales se puede trabajar, esto son `iterrrows()` e `.itertuples()`. A continuación se estudia una transformación sobre la columna `Preu` del dataset `df_m`.

En primer lugar, se llevan los datos al formato correcto

In [ ]:
df_m['Preu'] = df_m['Preu'].str.replace('$','')
df_m['Preu'] = df_m['Preu'].str.replace(',','')
df_m['Preu'] = df_m['Preu'].str.replace('?','0')
df_m['Preu'] = df_m['Preu'].astype('float32')
df_m['Preu'].head()

Se procede a normalizar los valores de `Preu` utilizando distintas técnicas de computo. Acá se comienza con un `for` de acceso eficiente por medio de `iat`:

In [ ]:
# Valores iniales
df_m.reset_index(inplace=True, drop=True)

m = df_m['Preu'].min()
M = df_m['Preu'].max()

n = len(df_m)

norm = lambda x: (x -m)/(M - m)

In [ ]:
%%timeit -r 10
#For con acceso eficiente

price = np.zeros(n)
for idx in range(n):
    price[idx] = norm(df_m.iat[idx, 1])

A continuación se efectua la misma operación utilizando `.iterrows()`. Este método genera un iterable a partir del dataframe sobre el que se opera. Cada elemento de este iterable es de la forma `(index, row)` donde `index` es el indice asociado a una fila y `row` el contenido de tal fila.

In [ ]:
%%timeit -r 10
# For con iterrows()
price_1 = np.zeros(n)
for idx, row in df_m.iterrows():
    price_1[idx] = norm(row[1])

En este caso se observa una ventaja substancial del método de indexado escalar y eficiente `iat`. 

Otro método de iteración alternativo es `.itertuples()` este se comporta de manera similar a `iterrows()` pero proporciona una *tupla nombrada* como objeto de iteración. Una tupla nombrada se comporta como un diccionario pero solo soporta la notación `tupla_nombrada.elem` para acceder al valor asociado a la llave `elem`. 

In [ ]:
%%timeit -r 10
# For con iterrows()
price_2 = np.zeros(n)
for row_tuple in df_m.itertuples():
    price_2[row_tuple.Index] = norm(row_tuple.Preu)

En comparación a los métodos anteriores se tiene una mejora considerable al momento de iterar. 

Para continuar y a modo de comparación en cuanto a los ordenes de magnitud en tiempos de ejecución, se procede a vectorizar utilizando el método `.map` (forma más básica de vectorizar en Pandas).

In [ ]:
%%timeit -r 10
price_3 = df_m['Preu'].map(norm)

Acá se evidencia que en efecto el tiempo de ejecución reduce en torno a la mitad. Finalmente, se vectoriza utilizando la notación recomendada

In [ ]:
%%timeit -r 10
price_4 = (df_m['Preu'] - m) / (M-m)

Donde el tiempo de ejecución es por lo menos un orden de magnitud más rápido. Por último, utilizando las capacidades de vectorización de NumPy, es posible aplicar la operación `norm` directamente sobre el arreglo asociado a los valores de `df_m['Preu']`.

In [ ]:
%%timeit -r 10
data = df_m['Preu'].values
norm(data)

Aquí se ve como la velocidad de ejecución mejora aún más, incluso cuando se incurre en operaciones de acceso de memoria por medio de 
`data = df_m['Preu'].values`.

**Ejercicios**

1. Al igual que NumPy, Pandas se beneficia de la libreria `numexpr`. Estudie la [documentación de Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html) al respecto y observe cuando es útil usar la expresión `eval()`.

## Cython

Cython es una extensión de Python, la cual permite declaración de tipo para funciones, variables y clases. Esto permite compilar rutinas de Python,las cuales actúan como código de *C*. 

Por definición, la sintaxis de Cython contiene a la de Python pero tiene la ventaja de ser compilada. Por lo general, los archivos de código Cython poseen la extensión `.pyx` y se construyen configurando la función `setup` del módulo `setuptools` (ver el ejercicio de esta sección). Por otra parte, la extensión de Jupyter `cython` hace esta tarea de manera automática, compilando el código de una celda determinada.

**Ejemplo**

Se crea una rutina básica de Cython y se compila por medio de la extensión `cython`. Para ello, se carga dicha extensión

In [ ]:
%load_ext cython

posteriormente, se define una función sencilla

In [ ]:
%%cython
def funcion_basica_cython(n=int(10e5)):
    s = 0
    for i in range(n):
        s += i**2 - i*(i-1)
    
    return s

se implementa la misma función pero utilizando unicamente Python

In [ ]:
def funcion_basica(n=int(10e5)):
    s = 0
    for i in range(n):
        s += i**2 - i*(i-1)
    
    return s

finalmente se comparan los tiempos de ejecución esperados

In [ ]:
%timeit funcion_basica()

In [ ]:
%timeit funcion_basica_cython()

Con lo anterior se observa una mejoria en el tiempo de ejecución, esta se logra unicamente pues el código ejecutado a sido compilado y no interpretado.

**Ejercicio**

El objetivo de este ejercicio es que compile una función de Cython utilizando el *procedimiento estándar*, para ello:

1. Cree un módulo de Python, este módulo contiene la función `func`, programe una rutina simple para dicha función utilizando sólo código de Python nativo. Guarde el módulo con el nombre `cython_func.pyx`

2. En la misma carpeta donde se encuentra el módulo `cython_func.pyx` escriba el programa `setup.py`. 

3. En `setup.py`importe la función `setup` del módulo `setuptools`.

4. En `setup.py`importe `cythonize` del módulo ` Cython.Build`. 

4. Configure el módulo a compilar, para ello, en `setup.py` llame la función `setup` entregando como argumento los parámetros `'name'`, `'ext_modules'` y `'zip_safe'`. Estudie el significado de estos en la documentación de Cython. 

5. Ejecute `setup.py ` desde la consola, para ello se debe ejecutar `python setup.py build_ext --inplace`.¿Cuál es la función de `build_ext`?

6. Importe su módulo y cuantifique su tiempo de ejecución en contraste con su versión sin compilar. 

### Tipos Estáticos

En Python se tienen variables con tipo *dinámico*, es decir, tales variables pueden cambiar de tipo de dato durante la ejecución de un programa, sin declararlo de manera explicita. Si bien esto hace que el lenguaje sea más felxible, produce una baja en el rendimiento al momento de interpretarse el código. 

Cython permite el uso de tipos de dato *estáticos*, lo que permite optimizar aún más las rutinas de Python. Para declarar un tipo de dato en Cython, se utiliza la orden `cdef`, está puede ser utilizada en variables y funciones. 

Para su uso en **variables**, se hace uso de `cdef` y del tipo de dato que se busca declarar. 

**Ejemplo**

Para declarar la variable `j` con un tipo de dato entero (estático) se puede ejecutar el siguiente código de Cython:

```Cython
cdef int j 
```

**Ejercicio**

1. Defina la variable `d` , esta debe ser númerica de punto de flotante con doble precisión. Haga esto desde Cython.

2. Asigne un valor con tipo de dato `string` a la variable `d`. ¿Se diferencia el comportamiento de esta asignación con el comportamiento de Python nativo?

Se procede a estudiar la ventaja en rendimiento que ofrecen los tipos de dato estáticos, para ello, se define la siguiente función

In [ ]:
def dynamic_func(n=int(10e5)):
    '''Funcion sencilla con variables dinamicas.'''
    
    s = 0
    for i in range(n):
        s += i**2 - i - 1
    
    return s

Se prueba su eficiencia

In [ ]:
%timeit dynamic_func()

Se define la misma función usando Cython y se mide el tiempo de ejecución

In [ ]:
%%cython
def dynamic_func_cython(n=int(10e5)):
    '''Funcion sencilla con variables estaticas preprocesada por Cython.'''
    
    s = 0
    for i in range(n):
        s += i**2 - i - 1

    return s

In [ ]:
%timeit dynamic_func_cython()

Finalmente se declaran tipos de dato estáticos y se mide el tiempo de ejecución

In [ ]:
%%cython
def static_func_cython(n=int(10e5)):
    '''Funcion sencilla con variables estaticas.'''
    
    cdef int i, s=0
    
    for i in range(n):
        s += i**2 - i - 1

    return s

In [ ]:
%timeit static_func_cython()

Con lo que se observa una mejora substancial en el rendimiento. 

**Ejercicio**

Cython se relaciona intimamente con *C*, por tal motivo, es posible realizar algunas operaciones de tal lenguaje de manera nativa en Cython. 

1. Defina la variable `x=1` estática tipo *int* en cython. Defina la variable `y` estática tipo `double` en cython. Finalmente ejecute `y = <double> x`. ¿Qué efecto tiene el comando `<double>` en la orden anterior?

2. El método anterior se denomina *cast* y es nativo en *C*. Investigue otros métodos de *casting* de potencial utilidad en Cython.

3. Declare la variable `obj = 'ejercicio'` de tipo estático `object`. Ejecute la orden `obj = 1`.¿Se espera algún tipo de error en la ejecución?¿utilizar este tipo de datos mejora el rendimiento en comparación a trabajar con tipos de datos dinámicos?

Cython permite declarar tipos de datos estáticos más especificos para **funciones**, en este caso, las funciones declaradas con datos estáticos funcionan de la misma manera que una función nativa de Python, con la excepción de que se realiza un revisión de argumentos (*type checking*) al ser ejecutadas. La sintaxis a utilizar es:

```cython
cdef res_dtype func(dtype_1 var_1, ..., dtype_n var_n):
    res = do_stuff(var_1,var_2,..., var_n)
    return res
```

En este caso `res_dtype` corresponde al tipo de dato esperado como resultado de la función, mientras que `dtype_1, dtype_2, ..., dtype_n` corresponden los tipos de dato esperados como input de la función. Las funciones definidas por este procedimiento son transformadas a código *C* pero **no pueden ser accedidas desde Python**, por lo tanto, para utilizarlas, se deben llamar dentro de scripts de Cython (archivos `.pyx` ejecutados por medio de la orden `cython`). Una manera de resolver el conflicto anterior y poder acceder a funciones de Cython desde Python, es por medio de la creación de módulos de Cython.

**Ejemplo**

Se implementa un módulo de Cython, este posee sólo una función

In [ ]:
%%file func_module.pyx
def func(x):
    return x**3 + 2*x**2 + 3*x + 4

Se construye el archivo de configuración correspondiente y se guarda en `setup.py`.

In [ ]:
%%file setup.py
from setuptools import setup
from Cython.Build import cythonize

setup(
    name='modulo test de funciones',
    ext_modules=cythonize("func_module.pyx")
)

Finalmente, se construye el módulo de Cython, para ello, se ejecuta `python setup.py build_ext --inplace` en la consola. 

In [ ]:
#En maquinas con windows este script puede no funcionar
!python setup.py build_ext --inplace

Con lo anterior se crea un archivo `.c` con el nombre del módulo. Finalmente para trabajar con el módulo creado, se importa de manera estándar por medio de:

In [ ]:
from func_module import func

se estudia su tiempo de ejecución

In [ ]:
%timeit func(10)

Se compara el tiempo de ejecución con el de la implementación que utiliza Python directamente

In [ ]:
def func_python(x):
    return x**3 + 2*x**2 + 3*x + 4

In [ ]:
%timeit func_python(10)

Con lo que se aprecia una mejora en el rendimiento al usar Cython.

Otra forma de acceder a funciones de Cython desde Python, es por medio del comando `cpdef`. Al utilizarlo, Cython genera 2 versiones de la función a definir, una accesible desde Cython y otra accesible desde Python, esta última, se carga en el entorno de Python al ser definida. La sintaxis de `cpdef` es equivalente a la de `cdef`.

```cython
cdef res_dtype func(dtype_1 var_1, ..., dtype_n var_n):
    res = do_stuff(var_1,var_2,..., var_n)
    return res
```

Se define la función anterior por medio de `cpdef` y se prueba su rendimiento

In [ ]:
%%cython
cpdef func_cpdef(x):
    return x**3 + 2*x**2 + 3*x + 4

In [ ]:
%timeit func_cpdef(10)

aqui se observa un comportamiento analogo al de importar una función definida por `cdef`. Finalmente se crea una verisón con tipos de dato estáticos

In [ ]:
%%cython
cpdef double func_cython(double x):
    return x**3 + 2*x**2 + 3*x + 4

In [ ]:
%timeit func_cython(10)

En este último caso se observa un mejora notoria en el rendimiento. 

Con respecto a la relación entre `def`, `cdef` y `cpdef` es bueno tener en cuenta:

1. Las funciones definidas por `def` son llamadas por medio de Python y por tanto interpretadas, lo que las hace inherentemente más lentas. Sin embargo, son extremadamente flexibles pues operan con objetos genéricos. 

2. La funciones definidas con `cdef` son traducciones de Python a *C*. Sus tipos deben ser declarados para ganar un rendimiento significativo. Por ser funciones de *C*, las funciones `cdef` no son visibles de manera nativa por Python.

3. Las funciones definidas por medio de `cpdef` generan funciones `def` y `cdef` adjuntas. En este caso la función `def` esta enlazada directamente a la `cdef` y puede ser ejecutada desde Python.

Por lo anterior, es recomendable optimizar utilizando funciones `cdef` (o `cpdef` si se desea) cuando se incurre en cálculos que debe ser realizados múltiples veces, más aún, en tales funciones se alcanza máxima eficiencia cuando se declaran tipos de dato estáticos. 

Dentro de **clases**, se pueden definir funciones y variables estáticas utilizando `cdef`. En este caso, se utiliza la sintaxis:

```cython
cdef class ClaseEstatica:
    # Tipos de datos estaticos
    cdef dtype_1 var_1
    ...
    cdef detype_n var_n
    
    # Metodos de Python 
    def normal_method(self,*args,**kwargs):
        do_stuff(self,*args,**kwargs)
        
    # Metodos de Cython
    cpdef res_dtype cython_method(self,*args,**kwargs):
        do_stuff(self,*args,**kwargs)
    
```

Las clases de Python pueden heredar de clases `cdef` pero no se tiene la relación reciproca. Por otra parte, Cython sólo permite herencia simple. 

**Ejemplo**

Se define una estructura clases en Cython, se almacena dicha estructura en un módulo de Cython para asegurar su disponibilidad. En primer lugar se define la clase `Function`

In [ ]:
%%cython
cdef class Function:
    '''Objeto que abstrae el concepto de funcion'''
    
    cpdef double apply(self, double x):
        return 1

Luego se define una clase que hereda de `Function` y anula el método de Cython `.apply()`, para que Jupyter reconozca la relación de herencia es necesario definir todas las dependencias en la misma celda.

In [ ]:
%%cython
cdef class Function:
    '''Objeto que abstrae el concepto de funcion'''
    
    cpdef double apply(self, double x):
        return 1
    
cdef class Poly(Function):
    '''Funcion polinomial.'''
    
    cdef int d
    
    def __init__(self, int d):
        self.d = d
        
    cpdef double apply(self, double x):
        return sum([x**d for d in range(self.d)])

Se observa que la clase `Poly` se inicializa y opera como corresponde

In [ ]:
p = Poly(5)
print('Resultado de p(5):', p.apply(1))

No obstante, si se desea acceder al atributo `.d`, no es posible

In [ ]:
try:
    print('El valor de p.d es: ', p.d)
        
except AttributeError:
    print('No fue posible acceder a p.d')

Para declarar atributos públicos en Cython se utiliza la orden `public`. 

In [ ]:
%%cython
cdef class Function:
    '''Objeto que abstrae el concepto de funcion'''
    
    cpdef double apply(self, double x):
        return 1
    
cdef class Poly(Function):
    '''Funcion polinomial.'''
    
    # Se agrega la orden public
    cdef public int d
    
    def __init__(self, int d):
        self.d = d
        
    cpdef double apply(self, double x):
        return sum([x**d for d in range(self.d)])

Se intenta acceder nuevamente a `p.d`

In [ ]:
p = Poly(5)

try:
    print('El valor de p.d es: ', p.d)
        
except AttributeError:
    print('No fue posible acceder a p.d')

Con lo que se logra lo buscado. 

Las clases de Cython se conocen también como *tipos extendidos*, esto quiere decir, que al utilizar `cdef class` se está creando un nuevo tipo de dato. Utilizando el código anterior, se agrega una función que toma objetos tipo `Poly` y entrega una transformación de estos

In [ ]:
%%cython
cdef class Function:
    '''Objeto que abstrae el concepto de funcion'''
    
    cpdef double apply(self, double x):
        return 1
    
cdef class Poly(Function):
    '''Funcion polinomial.'''
    
    # Se agrega la orden public
    cdef public int d
    
    def __init__(self, int d):
        self.d = d
        
    cpdef double apply(self, double x):
        return sum([x**d for d in range(self.d)])
    
#Se utiliza return void pues la funcion no retorna valores,
cpdef void transform(Poly p):
    p.d += 2

Observe que el tipo de dato retornado es *void* (la función no retorna valores) y el tipo de dato input es *Poly*. Se prueba el funcionamiento del procedimiento anterior

In [ ]:
p = Poly(5)
print('Valor de p.d antes de transformar:', p.d)
transform(p)
print('Valor de p.d despues de transformar:', p.d)

### Trabajo con módulos de Cython

Al trabajar con módulos, es necesario diseñar la organización de las funciones y clases implementadas. Los módulos de Cython no son lo excepción. Dentro de sus características, se tiene la existencia de *archivos de definición* o *header files*, similares a librerias de *C*. Por otra parte, la interacción entre módulos de Cython se rige por un comportamiento más bien similar al de Python por medio del comando `cimport`. 

Los archivos *header* de Cython tienen extensión `.pxd` y se utilizan como una interfaz entre Cython  código de C. La idea de estos archivos es servir de enlace entre las *declaraciones* ,*prototipos* o definiciones de funciones y tipos de datos con su implementación en *C* (o en un archivo `.pyx`)

**Ejemplo**

Se construye un módulo de Cython para las funciones promedio aritmético y geométrico de dos números . En primera instancia se definen las funciones

In [ ]:
%%file header_example.pyx
cdef double mean(double x, double y):
    return (x+y)/2

cdef double geo_mean(double x, double y):
    return pow(x*y,0.5)

Para compartir las definiciones de `mean` y `geo_mean`, se genera un archivo *header*, observe que solo contiene las definiciones de las funciones involucradas

In [ ]:
%%file header_example.pxd
cdef double mean(double x, double y)
cdef double geo_mean(double x, double y)

Según el método anterior, se puede importar el módulo `header_example` en cualquier rutina de Cython por medio de `cimport`, se construye la rutina `medias.pxy` que recibe 2 números y entrega una transformación entre su promedio arimético y geométrico.

In [ ]:
%%file transform.pyx
from header_example cimport mean, geo_mean

cpdef double transformer(double x,double y):
    return (mean(x,y)**2 - geo_mean(x,y))*0.5

Finalmente para ejecutar la función `transformer`, es necesario configurar el módulo correspondiente.

In [ ]:
%%file setup_3.py
from setuptools import setup
from Cython.Build import cythonize

setup(ext_modules=cythonize(["header_example.pyx", "transform.pyx"]),
     zip_safe = False)

In [ ]:
!python setup_3.py build_ext --inplace

Se verifica el correcto funcionamiento al importar la función `transformer`. Observe que es posible utilizar dicha función desde Python, pues fue definida por medio de `cpdef`.

In [ ]:
from transform import transformer
transformer(1,2)

### Arreglos de Cython, Numpy y C

Una de las grandes ventajas de *C* en cuanto a rendimiento es su manejo de arreglos. Si bien estos son rápidos debido a su *bajo nivel* presentan un gran desafio en su manejo, pues requieren de especificaciones en cuanto a tipos de dato y espacios de memoria (por medio de direcciones y punteros). No obstante, Cython provee una manera sencilla de interactuar con estas estructuras. 

Los arreglos *C* son secuencias contenedoras de bajo nivel. Para comprender estas secuencias, es necesario comprender el significado del término *memory address* o *dirección de memoria*. En *C* una dirección de meoria corresponde a la ubicación del espacio de memoria utilizada por una estructura o variable, se denota por el símbolo `&`. 

**Ejemplo**

Si se declara una variable tipo `float` de precisión simple, se posicionan 32 bits de memoria, la posición en la cual se encuentra la información de la variable (donde se encuentran esos 32 bits) es su dirección de memoria. Para obtener acceso a tal dirección, se utiliza el símbolo `&`. 

**Obs**: Para imprimir valores en pantalla usando *C*, se utiliza la función `printf` del módulo `libc.stdio`. No es posible usar la función nativa `print` de Python en este caso, pues no puede convertir direcciones de memoria a objetos interpretables. Por otra parte, los resultados de `printf` son del tipo `std output` o *salida estándar*, este tipo de salidas son las que se imprimen en la consola, lamentablemente IPython (y por tanto Jupyter) no interactúa con outputs de tipo estándar de manera nativa. Por lo anterior, se utiliza la librería `wurlitzer` que captura salidas estándar asociadas a un programa de Python y las imprime en la consola de IPython.

In [ ]:
%%cython
from wurlitzer import sys_pipes
from libc.stdio cimport printf

cdef float a 

with sys_pipes():
    printf("%p",&a)

El resultado de esta operación es la dirección de memora de la variable `a`de tipo `float`. Para almacenar direcciones de memoria, se puede hacer uso de una estructura especial denominada *puntero* o *pointer* en inglés. Estos se declaran usando `*` como prefijo. Si se quisiera almacenar la dirección de memoria de la variable `a`, se debe declarar un puntero de tipo `float`, esto se hace de la siguiente manera

In [ ]:
%%cython
cdef float a
cdef float *b
b = &a 

En este caso `b` es un puntero que hace referencia a la dirección de memoria de la variable `a`. Si se quiere acceder a la información ubicada en la dirección de memoria referenciada por un puntero `p` se utiliza la notación `p[0]`.

**Obs**: Si bien acceder usando `p[0]` es un comportamiento aceptado en *C*, la manera habitual de hacerlo es por medio del operador de *deferencia* `*`, en algunas implementaciones de Cython, usar dicho operador puede generar problemas.

In [ ]:
%%cython
from wurlitzer import sys_pipes
from libc.stdio cimport printf

cdef float a
cdef float *b = &a

a = 2.0

with sys_pipes():
    printf("Valor de &a: %p \n", &a)
    printf("Valor de b : %p \n", b)
    printf("Valor de a : %f \n", a)
    printf("Valor de *b: %f \n", b[0])

Con lo anterior se deduce que `*p = &var` hace que la dirección de memoria de `var` se guarde en la variable tipo *puntero* `p`. El valor asociado a la variable `var` será almacenado en `p[0]` si `var` es una constante. 

El caso general corresponde a declarar arreglos de *C*, en este proceso, el programa posiciona una cantidad de espacio de memoria solicitada. Por ejemplo, se se desea crear un arreglo de datos `float` con 5 elementos, se reserva el espacio correspondiente a 4 bytes (o 32 bits) para cada uno de los 5 elementos, este espacio (de 40 bytes en total) consite de bloques contiguos de memoria. 

En cython es posible declarar arreglos de *C* por medio de 
```cython
cdef dtype arr[n]
```
donde `dtype` es el tipo de dato contenido, `arr` es el nombre del arreglo y `n` la cantidad de elementos solicitados. 

**Ejemplo**

Se declara un arreglo consistente en 10 arreglos de 3 elementos `float`.

In [ ]:
%%cython
cdef float matrix[10][3]

En este caso, la memoria se posiciona fila por fila, de esta manera, si la primera fila de `matrix` es el arreglo `[0,1,2]` y la segunda `[33,44,55]`, la primera sección de memoria asociada a `matrix` será aquella que corresponde a los elementos 0,1,2,33,44,55,..., hasta agotar las filas de `matrix`. Finalmente, `matrix[0]` hace referencia al primer arreglo `[0,1,2]`, mientras que `matrix[1]` al segundo y así sucesivamente. 

Es conveniente pensar en los arreglos multidimensionales, como arreglos de arreglos, en este caso, se tienen 10 arreglos de 3 elementos, pero no hay restricción en la cantidad de *niveles* a declarar.

Lo comentado anteriormente tiene una consecuencia pŕactica, y es que cuando se itera sobre arreglos, es necesario tener en cuenta sobre qué dimensión es más eficiente iterar. Esto pues, si por ejemplo en `matrix` recorremos por la primera dimensión, se accede a elementos *no contiguos* en memoria lo cual hace que el acceso a los elementos sea más lento. Por otra parte, si se itera sobre la última dimensión, siempre se accede a elementos contiguos en memoria. 

**Ejercicio**

1. Investigue como se almacenan arreglos de más de 2 dimensiones en *C*.

Los arreglos de *C* no soportan *slicing* pero se accede a sus elementos utilizando una notación similar a la Python.

In [ ]:
%%cython
from itertools import product as prod

cdef int matrix[10][3]
cdef int i,j

for i,j in prod(range(10),range(3)):
    matrix[i][j] = i + j

print('Elemento matrix[0][0]: ', matrix[0][0])
print('Elemento matrix[4][2]: ', matrix[4][2])

Existe una estrecha relación entre punteros y arreglos. En arreglos, de hecho, la variable asociada al arreglo en realidad apunta a la dirección de memoria asociada a su primer elemento. 

En el siguiente código se genera un arreglo de dos dimensiones (arreglo de arreglos), se crea un puntero hacia su dirección de memoria. 

In [ ]:
%%cython
from libc.stdio cimport printf

from wurlitzer import sys_pipes
from itertools import product as prod

cdef int i, j
cdef int matrix[10][3]
cdef int (*p)[10][3]

p = &matrix

for i,j in prod(range(10),range(3)):
    matrix[i][j] = i + j

with sys_pipes():
    printf("%p \n", matrix)
    printf("%p \n", matrix[0])
    printf("%p \n", &matrix[0][0])
    printf("%p \n", p[0])
    printf("%p \n", p[0][0])

Con lo anterior se confirma que la variable `matrix` apunta a la dirección de memoria de su primer elemento `matrix[0]` que a la vez es un arreglo de tres dimensiones, el cual apunta a la dirección de memoria de su primer elemento `&matrix[0][0]`. Esto es equivalente a definir un puntero `p` que almacena la dirección de memoria `&matrix` y acceder a su primer elemento `p[0]` que es un puntero asociado a `&matrix[0][0]` y que por tanto se puede acceder por medio de `p[0][0]`.

El uso de arreglos y punteros de *C* es comendado cuando se busca utilizar librerías de *C* en conjunción con rutinas de Python.

### Arreglos de Numpy en Cython

Los arreglos de NumPy se pueden utilizar como objetos normales de Python en Cython, la ventaja de esto, es que se hereda el rendimiento de los métodos de *broadcasting*. 

Cython posee un módulo NumPy propio, la oportunidad que esto ofrece, es evitar operaciones realizadas por el interprete de Python en operaciones de acceso a arreglos. 

Los arreglos Numpy pueden ser declarados con el tipo de dato `ndarray` dentro de Cython, para ello, es necesario importar `numpy` por medio de `cimport`.

**Ejemplo**

Se importa `numpy` en Cyhton y se declara el arreglo `vec` utilizando el tipo de dato `ndarray`. El arreglo`vec` será unidimensional y contendrá elementos de tipo `double`. Para declararlo, se utiliza:

```cython
cdef ndarray[double, ndim=1] vec
```

La notación del tipo `cdef ndarray[dtype, ndim] var`, que hace uso de paréntesis `[]` se denomina *buffer sintax*, este tipo de sintaxis no se permite en el *scope* global de un módulo de Cython por lo que ejecutar directamente dicha orden produce un error. Para solucionar dicho problema, se crea un nuevo *scope* por medio de una función con la cual se accede a la variable.

A continuación se crea una función que opera sobre un arreglo de NumPy por medio de Cython

In [ ]:
%%cython
cimport numpy as np_c
import numpy as np

cpdef np_c.ndarray[double, ndim = 1] vec_func(int d):
    
    cdef np_c.ndarray[double, ndim = 1] vec
    vec = np.zeros((d,), dtype = 'float')
    
    cdef int i
    for i in range(d):
        vec[i] = 1 - 2*i + i**2 
        
    return vec

Como la función es definida por medio de `cpdef` se puede acceder a ella desde Python. Procedemos a calcular su tiempo de ejecución promedio

In [ ]:
%timeit vec_func(100)

Se compara con la misma operación pero vectorizada en Python

In [ ]:
import numpy as np

def python_vec(d):
    h = np.arange(d, dtype = 'double')
    return h**2 -  2*h + 1

In [ ]:
%timeit python_vec(100)

Lo anterior demuestra que el manejo de arreglos numpy directamente desde Cython ofrece un mayor rendimiento.
No obstante, existe una mejor manera de vectorizar el código de Python y obtener rendimeintos similares a los de Cython.

**Ejercicios**

1. Observe que la relación `vec[i] = 1 - 2*i + i**2` puede resumirse en `h**2` para cierto objeto NumPy `h`. Utilice este observación para modificar el código de la función `python_vec`. Compare el tiempo de ejecución de su función con la implementación de Cython. *Hint* Su implementación será un poco más lenta que `vec_func` pero más rápida que `python_vec` (x4 aproximadamente) pero tendrá la ventaja de ser mucho más sencilla que  `vec_func`.

2. Genere una versión en Cython de su función vectorizada. Se espera que esta función sea más lenta que la versión de Python. ¿A qué se debe esto?

Las funcionalidades que ofrece el módulo de NumPy de Cython son más bien acotadas y su notación reduce la legibilidad del código. Sumado a lo anterior, es posible que no se tengan mayores ganancias con respecto al uso de *broadcasting*, por esto motivos, se recomienda su uso sólo en circunstancias especificas que requieran un manejo de memoria especial. En casos más generales, se recomienda el uso de **memoryviews** estáticas.

Una vista de memoria,**memoryview** estática o *typed memoryview* es un objeto similar a los arreglos de *C* y NumPy pues operan en zonas de memoria contigua. Este tipo de objeto, mantiene una referencia a zonas especificas de memoria pudiendo leer y modificar sus contenidos. Se puede definir una vista de memoria para objetos multidimensionales (arreglos de arreglos de arreglos ...) por medio de la sintaxis:

```cython
cdef dtype[:,:,:] var
```

Donde `dtype` es el tipo de dato a contener, `[:,:,:]` hace referencia a la cantidad de dimensiones (en este caso 3) y `var` es el nombre la variable asociada.

**Ejemplo**

Se genera un arreglo de NumPy y se almacena en una vista de memoria.

In [ ]:
%%cython
import numpy as np

cdef int[:] vec

np_vec = np.zeros(100, dtype = 'int32')
vec = np_vec

print('Acceso :',vec[0])
vec[0] = 1

print('Modificacion :', np_vec[0])

Con lo anterior, se tiene que `vec` puede acceder y modificar la información de `np_vec`, sin embargo es sólo una *vista* o *puntero* si se desea a su información contenida. Las vistas de memoria operan por tanto de la misma manera que un *slice* de NumPy en Python (recordar mutabilidad de los arreglos de NumPy) y de hecho heredan su notación de acceso.

In [ ]:
%%cython
import numpy as np

cdef int[:] a
a = np.arange(1,10, dtype = 'int32') + 1

# Se accede a los elementos pares de a
print('Slices de a: \n ')

cdef int i
for i in a[::2]:
    print(i)

**Ejercicio**

1. Es posible copiar vistas de memoria por medio de una asignación simple en Cython. Defina dos vistas de memoria, `a` de dimensión 2 y `b` de dimensión 1. Asigne un arreglos de NumPy a las vistas `a` y `b`. Finalmente reemplace una dimensión de `a` con el valor de `b`. ¿Afecta este procedimiento a los arreglos de NumPy originales?

### Cython con Jupyer Notebook y Perfilamiento 

La optimización de código Cython requiere de comprobaciones exhaustivas, jupyter facilita tal análisis por medio del comando mágico `%%cython`, este comando toma opciones de linea como `-a`, con la cual es posible generar código de Cython anotado.

**Ejemplo**

Se produce una rutina de Cython anotada

In [ ]:
%%cython -a 
import numpy as np
cimport numpy as np_c

cdef double mean(double x, double y):
    return (x+y)/2

cdef double geo_mean(double x, double y):
    return pow(x*y,0.5)

cpdef double bench_p(int d):
    
    cdef  double[:,:] a
    cdef double res 
    
    a = np.random.rand(d, 2)
    res = sum([mean(pair[0],pair[1])-geo_mean(pair[0],pair[1]) for pair in a])
    
    return res

El código anterior genera una vista HTML del código original, donde las lineas amarallias indican el grado de interacción con Python, mientras más claro el nivel de color, menos interacción se tiene con Python y por tanto se tiene mayor velocidad. El extremo es un código sin lineas amarillas, esto corresponde a un código de *C*. 

A modo de ejemplo se estudian las lineas amarillas 7 y 8:

```cython
cdef double geo_mean(double x, double y):
    return pow(x*y,0.5)
```
En este caso, Cython nos indica iteracción con Python, esta ocurre pues se hace uso de la función `pow` nativa de Python. Se puede hacer el código más eficiente al reemplazar `pow` por su equivalente de *C*, esto se logra por medio de

```cython
from libc.math cimport pow
```

In [ ]:
%%cython -a 
import numpy as np
cimport numpy as np_c

from libc.math cimport pow
        
cdef double mean(double x, double y):
    return (x+y)/2

cdef double geo_mean(double x, double y):
    return pow(x*y,0.5)

cpdef double bench_c(int d):
    
    cdef  double[:,:] a
    cdef double res 
    
    a = np.random.rand(d, 2)
    res = sum([mean(pair[0],pair[1])-geo_mean(pair[0],pair[1]) for pair in a])
    
    return res

Con la adición anterior, vemos que se pierden las lineas amarillas asociadas a `geo_mean`. Se hace una comparación entre ambas implementaciones:

In [ ]:
%timeit -r 5 bench_p(10000) 

In [ ]:
%timeit -r 5 bench_c(10000)

Se observa que en general se gana un poco de eficiencia. A continuación se procede a cambiar las lineas 17 y 18:

```cython
a = np.random.rand(d, 2)
res = sum([mean(pair[0],pair[1])-geo_mean(pair[0],pair[1]) for pair in a])
```

En el caso de la linea 17, generamos el arreglo aleatorio por medio de 

```cython    
cdef int i;
for i from 0 <= i < d by 1:
    a[i][0] = rand()
    a[i][1] = rand()
```

Donde `rand`es la función generadora de números aleatorios de *C*, esta se importa por medio de:

```cython
from libc.stdlib cimport rand
```
Se define además la variable global `d` por medio de `DEF d  = 10000` y se solicita el espacio necesario para el arreglo `a` utilizando tal variable:

```cython
 cdef double a[d][2]
```

Vale destacar que se utiliza la notación *Pyrex* de ciclo for `for i from 0 <= i < d by 1:` que permite escribir ciclos `for` de Cython como código nativo de *C*. Finalmente se reemplaza la linea 18 por su correspondiente de *C* y se fusiona con el llenado de valores de `a` por medio de:

```cython

    cdef int i;
    cdef double res = 0
  
    for i from 0 <= i < d by 1:
        a[i][0] = rand()
        a[i][1] = rand()
        res = res + mean(a[i][0],a[i][1]) - geo_mean(a[i][0],a[i][1])
```

El código final tiene la siguiente forma:

In [ ]:
%%cython -a 
DEF d  = 10000

from libc.math cimport pow
from libc.stdlib cimport rand
        
cdef inline double mean(double x, double y):
    return (x+y)/2

cdef inline double geo_mean(double x, double y):
    return pow(x*y,0.5)

cdef double bench_cc():
    
    cdef double a[d][2]

    cdef int i;
    cdef double res = 0
  
    for i from 0 <= i < d by 1:
        a[i][0] = rand()
        a[i][1] = rand()
        res = res + mean(a[i][0],a[i][1]) - geo_mean(a[i][0],a[i][1])

    return res

Acá se tiene que la única parte que interactua con Python es la defición `cpdef` que como sabemos, genera un ejecutable de la función en el ambiente de Python, medimos el tiempo de ejecución:

In [ ]:
%timeit -r 5 bench_cc()

Vemos que se reduce aproximadamente a la mitad con respecto a las implementaciones anteriores. 

**Ejercicio**

1. En la definiciones de `mean` y `geo_mean` se utiliza el código `inline`. Investigue el significado de esta orden. *Hint*: la documentación de *C++* puede ser utilidad.

Para perfilar funciones de Cython en jupyter, se puede hacer uso de `%lprun`, sin embargo, para que esto funcione es necesario alterar las directivas de Cython por medio de los comentarios 

```cpython
# cython: binding=True
# cython: linetrace=True
```
Además de agregar la directiva de compilación `-f -c=-DCYTHON_TRACE=1`. Sumado a lo anterior, la función de referencia (función de benchmark) debe estar declara en Python usando `def`. Se ejecuta el código necesario para perfilar la función  `bench_cp`

In [ ]:
%%cython -f -c=-DCYTHON_TRACE=1
# cython: binding=True
# cython: linetrace=True
DEF d  = 10000

from libc.math cimport pow
from libc.stdlib cimport rand
        
cdef inline double mean(double x, double y):
    return (x+y)/2

cdef inline double geo_mean(double x, double y):
    return pow(x*y,0.5)

def bench_cp():
    
    cdef double a[d][2]

    cdef int i;
    cdef double res = 0
  
    for i from 0 <= i < d by 1:
        a[i][0] = rand()
        a[i][1] = rand()
        res = res + mean(a[i][0],a[i][1]) - geo_mean(a[i][0],a[i][1])

    return res

Finalmente se perfila usando `%lprun`

In [ ]:
%lprun -f bench_cp bench_cp()

## Compiladores

Cython permite ganar eficiencia por medio de la traducción y compilación de código Python. Sin embargo, la idea de compilar código de Python no es exlusiva de esta iniciativa. 

Un proyecto interesante la librería **Numba**, la cual en vez de traducir código Python a *C*, analiza y compila funciones de Python directamente. Compiladores como Numba, diseñados para compilar código en ejecución (y no previo a la ejecución) se denomina compiladores **JIT** (just in time). 

Numba permite compilar funciones individuales de Python usado una *máquina virtual de bajo nivel* o LLVM por sus siglas en inglés. LLVM es un conjunto de herramientas pensadas para escribir compiladores y no depende del lenguaje utilizado para programar. 

Por medio de LLVM Numba inspecciona funciones de Python y las compila utilizando una capa de representación intermedia similar a código *assembly*. La potencia de esta inspección radica en la inferencia de tipos de datos generando una versiones compiladas con tipos de datos estáticos.

Numba se basa principalmente en el decorador `@jit` con el cual se definen las funciones a compilar.

**Ejemplo**

Se genera una función simple y se compila usando el decorador `@jit`.

In [ ]:
def func(x):
    res = 0
    n = len(x)
    
    for i in range(1,n):
        res += x[i-1]*(x[i]-1)
        
    res /= n**3
    return res

Se prueba mide el tiempo de ejecución para arreglo

In [ ]:
n = int(1e5)
x = np.arange(n)

In [ ]:
%timeit func(x)

Se decora la función para ser compilada 

In [ ]:
from numba import jit

@jit
def func_n1(x):
    res = 0
    n = len(x)
    
    for i in range(1,n):
        res += x[i-1]*(x[i]-1)
        
    res /= n**3
    return res

se ejecuta la misma prueba de rendimiento

In [ ]:
%timeit func_n1(x)

Con lo que se observa un aumento substancial en la eficiencia. 

Por otra parte, se compara con la versión vectorizada en NumPy

In [ ]:
%timeit np.sum(x[:(n-1)]*(x[1:] -1))/n**3

con lo que se aprecia un aumento en rendimiento aún comparando con el código vectorizado de NumPy. En este caso, el aumento en el rendimiento viene dado por el manejo de memoria *inplace* que posee la función `func_n1` en comparación con el posicionamiento de un arreglo (temporal) extra de NumPy según la operación `x[:(n-1)]*(x[1:] -1)`. Por lo anterior, no se espera una mejora substancial al vectorizar `func_n1`.

In [ ]:
@jit
def func_n2(x):
    return np.sum(x[:(n-1)]*(x[1:] -1))/n**3

In [ ]:
%timeit func_n2(x)

Lo anterior confirma que no se aumenta el rendimiento de `func_n1` al vectorizar su comportamiento dado el manejo de memoria que requiere la operación vectorizada.

**Ejercicio**

1. Observe que `func_n1` puede operar sobre arreglos de Numpy o listas contenedoras números. Utilice `func_n1` sobre una lista de numeros y compare el rendimiento con una suma sobre una compresión de lista de Python. ¿Qué método es más rápido?

Numba permite indicar tipos de datos por medio de la firma o *signature* sobre la función decorada. La sintaxis es del tipo:

```python
@jit(output_dtype(input_dtype_1,..., input_dtype_n))
```
donde `output_dtype` es el tipo de dato esperado como respuesta de la función e `input_dtype_1`,...,`input_dtype_n` son los tipos de datos esperados como argumentos y proporcionados por Numba.

**Ejemplo**

Se observan la firmas de `func_n1` por medio del atributo `.signatures`

In [ ]:
func_n1.signatures

La firma anterior concuerda con el tipo de dato de `x`

In [ ]:
x.dtype

Se opera sobre `func_n1` utilizando un nuevo tipo de dato y se estudian nuevamente sus firmas de datos

In [ ]:
xx = np.random.rand(n).astype(dtype = 'float32')
func_n1(xx)
func_n1.signatures

Para agregar firmas para arreglos se utiliza la notación de *memoryview*. Se agrega una firma especifica a `func_n1`.

In [ ]:
from numba import float32

@jit(float32(float32[:]))
def func_n3(x):
    res = 0
    n = len(x)
    
    for i in range(1,n):
        res += x[i-1]*(x[i]-1)
        
    res /= n**3
    return res

Al especificar los tipos de datos operados, no se pueden entregar tipos de datos distintos pues se genera una excepción `TypeError  `. Por esto, se espera que `func_n3` funcione correctamente sobre `xx` pero no sobre `x`.

In [ ]:
try:
    print('func_n3(xx):',func_n3(xx))

except TypeError:

    print('El tipo de dato de x es', xx.dtype)
    print('Se espera como input float32') 

In [ ]:
try:
    print('func_n3(x):',func_n3(x))

except TypeError:
    print('El tipo de dato de x es', x.dtype)
    print('Se espera como input float32') 

**Ejercicios**

1. Numba permite especificar firmas de datos utilizando strings. Implemente una función de Python que retorne datos tipo `int32` y reciba como argumentos arreglos de tipo `float64`.

2. Implemente una función de Python y especifique por medio de Numba la firma para el input (sólo para input, no para el output).

3. Declare una función de Python y declare múltiples firmas para esta utilizando listas y formato string.

**Obs**: Se espera que especifique las firmas anteriores dentro de un decorador `@jit`.

Como se observó con Cython, es posible mejorar de manera substancial el rendimiento de ejecución en funciones compiladas por medio de la especificación de tipos de dato estáticos. Si bien el decorador `@jit` infiere de manera eficiente los tipos de datos (como se observa en `func_n1`) , existen casos donde este tipo de inferencia no es tan sencillo. 

Cuando Numba no es capaz de inferir los tipos de datos en una función de Python, se utiliza el interprete de Python en lo que se llama **Object mode** que por definición es menos eficiente que utilizar tipos de datos estáticos (inferidos o proporcionados) que se denomina **Native mode**. 

Para comprender la inferencia de tipos de datos hecha por Numba, se puede utilizar la función `inspect_types`. Esta entrega bloques formateados con información relevante sobre e manejo de variables, en el caso de la función `func_1` se tiene:

In [ ]:
func_n1.inspect_types()

Aquí se puede observar el proceso de inferencia asociado a la linea `res = 0`:
```
# --- LINE 14 --- 
    # label 0
    #   x = arg(0, name=x)  :: array(float32, 1d, C)
    #   $const2.0 = const(int, 0)  :: Literal[int](0)
    #   res = $const2.0  :: Literal[int](0)
    #   del $const2.0
    
    res = 0
```
En este caso se observa que la variable `res` es asociada a la constante ```const2.0 = const(int, 0)  :: Literal[int](0)``` que es inferida como una variable tipo `int`.

**Ejercicio**

La inferencia de tipos es una gran fortaleza de Numba, sin embargo, cuando esta falla se entra en *object mode* y se procede a calcular utilizando el interprete de Python. Esto termina generando un código más lento que el original.

1. Las listas anidadas generan problemas para determinar tipos de dato por parte de Numba. Compruebe esta afirmación.

## Paralelismo

El paralelismo se basa en el uso de múltiples unidades de computo de manera simulánea, con el el fin de mejorar la eficiencia en rutinas de código. La idea principal consite en enfrentar un problema de programación, dividiendolo en subunidades independientes y utilizar los núcleos disponibles de la máquina para resolver tales subunidades en paralelo.

Por lo general se necesitan téctinas de paralelismo en problemas de gran escala. Un problema donde todas sus subunidades son independientes entre si, se denomina **perfectamente paralelo**. Las operaciones elemento por elemento sobre arreglos poseen esta propiedad. 

Por lo general, las subunidades de un programa no son completamente independientes y necesitan compartir información, en estos casos,se debe tener en cuenta que la comunicación entre subunidades y los datos compartidos **quitan eficiencia** al problema que se resuelve, pues se incurre en *costos de comunicación*. La comunicación entre procesos es inherentemente costosa y puede llevar fallas de correctitud. Por ejemplo, al manejar de manera simultanea un arreglo, es posible que se corrompa su bloque de memoria asociado, por lo demás, si dicho bloque de memoria se encuentra en disco, su costo de acceso es bastante alto.

Por lo general, se enfrenta el problema de costo de comunicación y correctud del manejo de memoria por medio de sistemas que se comunican por medio de **memoria compartida** y **memoria distribuida**. En el caso de memoria compartida, las subunidades involucradas en el programa tienen acceso a un espacio común de memoria, este por lo general es de acceso rápido, si bien esto solventa el problema de velocidad de comunicación, el problema de correctitud sigue latente, por lo que se hace necesario utilizar técnicas para sincronizar los procesos sobre esta memoria. 

Por otra parte, el concepto de memoria distribuida concibe cada subunidad como un proceso completamente separado del resto con su propio espacio de memoria asociado. En este caso, la comunicación entre procesos se debe manejar de manera explicita y es más costosa que en el caso de memoria compartida, sin embargo, se reduce el riesgo de generar errores en el manejo de memoria. Este tipo de paralelismo se observa en *clusters* que consisten máquinas conformadas por múltiples unidades de computo independientes.

La manera usual en la que se implementan procesos de memoria compartida es por medio de **threads** o *hilos*. Estos consisten en subtareas originadas de un proceso en particular y que comparten recursos. 

Python puede manejar threads pero dado el diseño de su interprete, por defecto, se puede ejecutar solo una tarea a la vez, esto se conoce como **GIL** (Global Interpreter Lock). GIL provoca que cada vez que un hilo ejecute una orden de Python, se genere un bloqueo que solo será liberado una vez la ejecución del hilo termine. Esto hace que los hilos solo puedan ser ejecutados de manera secuencial.

Aunque GIL evita la ejecución paralela de las intrucciones de Python, es posible utilizar hilos mediante algunas librerías. La principal es `multiprocessing`

Multiprocessing ofrece una interfaz sencilla que incluye múltiples herramientas para manejar sincronozación y ejecución de tareas. Es posible importar esta librería de manera estándar. 

```python
import multiprocessing
```

Es posible crear procesos independientes por medio la clase `Process`, para ello basta extender el método `__init__` para inicializar los datos a procesar y generar el método `run` sobre el cual se ejecuta el proceso.

**Ejemplo**
 
Se genera un proceso independiente utilizando la clase `Process`

In [ ]:
from multiprocessing import Process
import time

class Proceso_ind(Process):
    
    def __init__(self, num):
        super().__init__()
        self.num = num
    
    def run(self):
        print('Proceso numero:', self.num)
        time.sleep(3)

Para utilizar el proceso se instancia un objeto de la clase `Proceso_ind` y se llama el método `.start()` 

In [ ]:
proc = Proceso_ind(5)
proc.start()

**Obs**:En el ejemplo anterior, no fue necesario utilizar el metodo anulado `.run()`, este es llamado por `.start()` de manera interna.

Las instrucciones luego de `proc.start()` y en general, luego de `Process.start` son ejecutadas de manera inmediata sin esperar a que el proceso finalice. 

In [ ]:
proc = Proceso_ind(5)
proc.start()
print('proceso siguiente')

Se comprueba que la ejecución de la orden siguiente es inmediata pues se debía esperar tres segundos dados por la orden `  time.sleep(3)`. En el caso en que se requiera esperar la finalización de un conjunto de tareas paralelas para luego recopilar resultados, es posible utilizar el método `.join()`.

In [ ]:
proc = Proceso_ind(5)
proc.start()
proc.join()
print('proceso siguiente')

Con la construcción actual, es posible levantar tantos procesos como se requiera, en esta caso se levantan 3 procesos.

In [ ]:
# Se definen los 3 procesos
proc = (Proceso_ind(1), Proceso_ind(2), Proceso_ind(3))

# Se mide el tiempo de ejecucion
start = time.time()

[*map(lambda p: p.start(), proc)]
[p.join() for p in proc]

end = time.time()


print('Tiempo de ejecución: ', end-start)

estos tres procesos corren de manera paralela, pues su tiempo de ejecución total es aproximado al tiempo de ejecución individual. Es necesario comprender que el orden de ejecución de procesos paralelos no es necesariamente ordenando y predecible pues depende de cómo el sistema operativo asigne los recursos. 

El módulo `multiprocessing` ofrece la clase `Pool`, esta permite manejar de manera sencilla un conjunto de procesos paralelos. Esta clase genera un conjunto de procesos llamados **workers** a los cuales se les asignan tareas por medio de los métodos `.apply()`, `.apply_async()`, `map` o `map_async`. 

El método `Pool.map()` actua de manera análoga la función nativa `map` de Python. Como resultado entrega una lista con los resultados, donde cada componente es el resultado de un worker de la clas.

**Ejemplo**

Para utilizar el método `.map` de la clase `Pool` se inicializa la clase, es posible hacerlo sin entregar un número de procesos asociados. Se genera también la función a paralelizar.

In [ ]:
from multiprocessing import Pool

def func(x):
    return x**2 - 1

p = Pool(3) #tambien funciona Pool()

Se comprueban los resultados y se cierra el conjunto de procesos por medio de `.close()`

In [ ]:
var = [2,4,6,8,10,12]
out = p.map(func, var)
p.close()

out

El método `.map_async()` es análogo al método `.map()` con la salvedad de que retorna un objeto tipo `AsyncResult`. Esto significa que el resultado de ejecutar `.map_async()` se obtiene de manera inmediata, pudiendo continuar con las demás ordenes que proceden pero seguirá calculandose como proceso de fondo. para acceder a los resultados asociados al objeto `AsyncResult` se utiliza el método `.get()`.

Utilizamos el método de mapeo asincrónico

In [ ]:
p = Pool(3) #tambien funciona Pool()

var = [2,4,6,8,10,12]
out = p.map_async(func, var)
out

accedemos a sus resultados

In [ ]:
print(out.get())
p.close()

**Ejercicios**

Los métodos `.apply()` y `apply_async()` son similares a `.map()` y `.map_async()`

1. ¿En qué se diferencian?
2. Programe una rutina que haga uso de `.apply()` y `apply_async()`. 

3. ¿Cómo se relaciona la clase `Pool` y los métodos de aplicación (`.map()`, `.apply()`, ...) con las funciones de Cython ?

El comportamiento predeterminado de `multiprocessing` es generar procesos con memoria independiente, sin embargo, permite definir ciertas variables en memoria compartida. Para definir una variabl en memoria compartida se utiliza la clase `Value`, a esta clase se le entrega un tipo de dato que puede ser `i` para entero, `f` para flotante, `d` para doble precisión entre otros. 

**Ejemplo**

Se define una variable en memoria compartida

In [ ]:
from multiprocessing import Value

comp_var = Value('d')
comp_var = 55

Al utilizar variables en memoria compartida se deben tener en cuenta los procesos que acceden a ella, manejando la *concurrencia*, es decir, si los procesos pueden acceder a dichas variables de manera simultanea u ordenada. Por lo general en la actualización de valores unidimensionales se debe tener en cuenta la concurrencia bloqueando el acceso simultaneo. En arreglos se puede permitir tal manipulación siempre que los computos sean independientes. 

Para bloquear el acceso a una variable compartida se hace uso de la clase `Lock`.

In [ ]:
from multiprocessing import Lock
lock = Lock()

A continuación se genera una rutina que accede a una variable de memoria compartida

In [ ]:
from multiprocessing import Process, Value

class Process_shared(Process):
    
    def __init__(self, var, n = 10000):
        super().__init__()
        self.var = var
        self.n = n

    def run(self):
        for i in range(self.n):
            self.var.value += 1

El proceso asociado toma un valor y le añade 1 hasta `n = 10000` veces por proceso. Se crea el valor inicial y se inicializan 3 procesos

In [ ]:
def test():
    var = Value('i')
    var.value = 0

    procs = [Process_shared(var) for i in range(3)]
    
    [p.start() for p in procs]
    [p.join() for p in procs]
    
    print(var.value)

Se prueba el resultado

In [ ]:
test()

Como se puede ver, el resultado no es necesariamente 30.000, esto se debe al acceso simultaneo y aleatorio de los procesos a `var`, para solucionar este problema se hace uso de `lock`, para ello se redefine la clase `Process_shared` observando que lock es un *context manager*

In [ ]:
class Process_shared_lock(Process):
    
    def __init__(self, var, n = 10000):
        super().__init__()
        self.var = var
        self.n = n

    def run(self):
        for i in range(self.n):
            with lock:
                self.var.value += 1

Se redefine la prueba asociada y se ejecuta:

In [ ]:
def test():
    var = Value('i')
    var.value = 0

    procs = [Process_shared_lock(var) for i in range(3)]
    
    [p.start() for p in procs]
    [p.join() for p in procs]
    
    print(var.value)
    
test()

Con lo cual se obtiene el resultado buscado

## Procesamiento Distribuido

El procesamiento distribuido hace referencia a la ejecución de tareas utilizando múltiples máquinas. Por lo general se refiere al trabajo con clusters de procesamiento y suele llevarse a cabo por medio de herramientas como MPI. 

En Python existen diversas librerías que permiten computación distribuida. En esta última sección estudiaremos una de ellas: Dask.

Dask permite escalar objetos y procedimientos de Python ya sea en un computador personal o un cluster de manera sencilla. Provee de funcionalidades para tratar, por medio de procesamiento multi-core, con datsets masivos que por lo general no caben en memoria. 

Dask proporciona planificadores de bajo nivel, cuya función es sincronizar tareas entre múltiples procesos o máquinas, análogo a la librería `multiprocessing` recientemente estudiada. 

Finalmente, Dask se instala de manera estándar por medio de `pip` o `conda` y se accede a sus objetos y funciones de la manera usual. 

### Paralelización de código con Dask

Una manera sencilla de utilizar Dask es comenzando por paralelizar rutinas de código, esto se puede hacer usando el decorador `delayed`.

**Ejemplo**

Se definen dos funciones y se paralelizan usando `delayed`

In [ ]:
from dask import delayed

import time
from time import sleep

def func_1(x):
    sleep(1)
    return x**2 + 1

def func_2(x, y):
    sleep(1)
    return x**2 + y**2 - 2

Se mide le tiempo de ejecución de las funciones implementadas, para ello se implementa un context manager:

In [ ]:
class Timer: 
    '''Context manager Timer'''
    def __enter__(self):
        self.inicio = time.time()
    
    def __exit__(self,*args,**kwargs):
        print('Tiempo de ejecución:', time.time() - self.inicio, 'segs') 

In [ ]:
with Timer() as t:
    x = func_1(5)
    y = func_1(55)
    z = func_2(x, y)

como es de esperar, se tiene un tiempo de ejecución similar a los 3 segundos esperados. Para paralelizar las funciones anteriores se hace uso del decorador `@delayed`, si recordamos que la notación:

```python
@decorator
def func(*args,**kwargs):
    res = do_stuff(*args,**kwargs)
    return res
```

Es de hecho equivalente a:

```python
func = decorator(func)
```

entonces se puede hacer una decoración *inline* de las funciones antes construidas, luego para parelelizar el código anterior, se hace uso del siguiente código:

In [ ]:
with Timer() as t:
    x = delayed(func_1)(5)
    y = delayed(func_1)(55)
    z = delayed(func_2)(x, y)

Donde se ve una mejora substancial en el tiempo de ejecución. El código de benchamark generado debería ejecutarse en aproximadamente 2 segundos, esto pues las operaciones que definen a `x` y a `y` son independientes y cada una toma un segundo, por lo que hacerlas de manera simultanea debe tomar alrededor de 1 segundo, finalmente, la sección que define a `z` toma 1 segundo y debe esperar el resultado de `x` e `y`, con lo que se tienen 2 segundos en total. Sin embargo, el código anterior se ejecuta en menos de un segundo, para entender este comportamiento haremos un print de pantalla con los resultados de la operación

In [ ]:
print('\n ---> Version secuencial')
with Timer() as t:
    x = func_1(5)
    y = func_1(55)
    z = func_2(x, y)
    print('z =',z)
    

print('\n ---> Version paralela')
with Timer() as t:
    x = delayed(func_1)(5)
    y = delayed(func_1)(55)
    z = delayed(func_2)(x, y)
    print('z =',z)

Con esto se observa que la versión secuencial se comporta como es debido, pero en la versión paralela lo que se obtiene es un objeto tipo `Delayed`. Lo que ocurre entonces es que cuando se opera con funciones `delayed` se procede a calcular de manera asincrónica, por lo tanto, si se desea obtener el resultado de `z` para continuar con nuevas secciones del código, es necesario recolectar los resultados antes de continuar, similar al método `.join()` de la librería `multiprocessing`, esto se hace por medio del método `compute`. Comparamos finalmente los resultados al incluir `compute` dentro del código paralelo

In [ ]:
print('\n ---> Version secuencial')
with Timer() as t:
    x = func_1(5)
    y = func_1(55)
    z = func_2(x, y)
    print('z =',z)
    

print('\n ---> Version paralela')
with Timer() as t:
    x = delayed(func_1)(5)
    y = delayed(func_1)(55)
    z = delayed(func_2)(x, y)
    print('z =',z.compute())

Con lo que vemos que se confirma la predicción en cuanto al tiempo de ejecución. 

Los objetos tipo de `Delayed` se evaluan de manera *lazy*, esto quiere decir que esperan a que todos los resultados intermedios estén realizados para evaluar sus operaciones. Al construir funciones `delayed` Dask genera un grafo acíclico dirigido (DAG) con el cual almacena las relaciones entre objetos `Delayed` intermedios, para visualizar las relaciones de objetos intermedios, asociados a un objeto `Delayed` se hace uso del método `.visualize()`.

In [ ]:
x = delayed(func_1)(5)
y = delayed(func_1)(55)
z = delayed(func_2)(x, y)

print('z =',z.compute())

In [ ]:
z.visualize()

**Ejercicio** 

El método de paralelización ofrecido por `delayed` puede ser de especial ayuda al momento de trabajar con ciclos, donde los resultados son recolectados en un arreglo y luego reducidos por alguna operación. 

1. Defina una función `f` que reciba un input numérico `x` y entrege como resultado una transformación de este. 

2. Por medio de ciclo `for` transforme los elementos de un arreglo de dimensión `n`, en cada iteración, guarde los resultados de la transformación en una componente de un arreglo.

3. Aplique una operación de reducción sobre el arreglo contenedor de resultados. (Ejemplo: el promedio de los elementos de arreglo contenedor)

4. Paralelice el proceso anterior usando el decorador `delayed` donde corresponda. ¿Qué funciones fue necesario decorar? ¿Cómo implementaría esto usando `multiprocessing`?




